In [73]:
# -*- coding: utf-8 -*-
# imports
import pandas as pd
import requests
import re
import os
from yaml import safe_load
from bs4 import BeautifulSoup as bs
import random


"""
sdg-csv-data-filler is the first module in a data pipeline to take
data from the SDG data repo and make it exportable as CSVW.
"""

# setting paths to directories and files
remote_data_url = "https://github.com/ONSdigital/sdg-data/tree/develop/data"
cwd = os.getcwd()
data_path = os.path.join(cwd, 'data')
out_path = os.path.join(cwd, 'out')
gap_filler_yam_path = (os.path.join
                       ("substitutions",
                        "gap_filler.yaml"))
header_mapping_yam_path = (os.path.join
                           ("substitutions",
                            "header_mapping.yaml"))
proxies = {'http': 'http://10.173.135.52:8080'}

In [103]:
def get_mapping_dicts(path_to_yaml):
    """
    Loads dictionaries for the gap filling and mapping of column names 
    from locally stored .yaml files
    
        Parameters:
            gap_filler_yaml (string): Path to the yaml file storing the values
                to fill gaps with in each column
            header_mapping_yaml (string): Path to the yaml file storing the 
                names to change headers to for each column
        Returns:
            dict: dict_from_yam
            
    """
    with open(path_to_yaml) as file:
        dict_from_yam = safe_load(file)
    
    return dict_from_yam

In [62]:
def find_csv_urls(url):
    """
    Provided with a data folder URL, this function finds the URLS
    of the CSV files within the folder. A generator is yielded with
    the links of all files in the folder.
        Parameters:
            url (string): the URL of the repo/folder which contains
                the CSV files to be captured
        Yields:
            string: generator, the next URL for the CSV file in the 
            remote data folder 
    """
    page = requests.get(url, verify=False) #proxies=proxies
    soup = bs(page.text, 'html.parser')
    csv_link_pattern = r"\/ONSdigital\/sdg-data\/blob\/develop\/data\/indicator_\d-\d{1,2}-\d{1,2}\.csv"
    to_repl_pattern = r"\/sdg-data\/blob\/develop"
    replacement_pattern = "/sdg-data/develop"
    list_of_links = []
    for link in soup.findAll('a', attrs={'href': re.compile(csv_link_pattern)}):
        link = link.get('href')
        link = re.sub(to_repl_pattern, replacement_pattern, link)
        yield ("https://raw.githubusercontent.com"+link)
        # Alternative to return a list
        # list_of_links.append("https://raw.githubusercontent.com"+link)
    # return list_of_links

In [197]:
def csvs_to_pandas(url):
    """
    Provided with a URL of a file, the fucntion will check if the CSV
    is populated and if not empty return a Pandas dataframe of the CSV
        Parameters:
            url (string): the URL of a CSV file to be captured
        Returns:
            pd.DataFrame: a Pandas dataframe of the CSV
    """
    if "no data for this indicator yet" in str(bs(requests.get(url).text)):
        return None
    else:
        return pd.read_csv(url)

In [86]:
def csvsample_to_pandas(path_to_file, pct=1.0):
    """
    A function to create a sample extract of a csv as a dataframe
    
        Parameters:
            path_to_file (string): full path to csv file
            p (float): decimal amount of lines to extract
            
        Returns:
            pd.Dataframe
            """
    p = pct/100  
    # keep the header, then take only 1% of lines
    # if random from [0,1] interval is greater than 0.01 the row will be skipped
    df = pd.read_csv(
             path_to_file,
             header=0, 
             skiprows=lambda i: i>0 and random.random() > p)
    return df


In [64]:
def fill_gaps(pd_df, gap_filler_dict):
    """
    Given a Pandas dataframe and a dictionary containing the column names
    the correct 'fillers' for each column, this function will fill
    each column with the correct values when empty cells are found.
        Parameters:
            pd_df (pd.Dataframe): the variable to which the dataframe 
                containing the csv data is assigned
            gap_filler_dict (dict): a dictionary with column name and value 
                to fill gaps as key value pairs, e.g.
                {"Age":"All","Sex":"T"}
        Returns:
            pd.Dataframe: A dataframe with all cells full"""
    df = pd_df.fillna(gap_filler_dict, axis=1)
    return df

In [198]:
def standardise_cell_values(pd_df, dict_of_nonstandard_standard):
    """
    Maps non-standard values e.g. "Males" to standard values like "M".
    Mapping is carried out on a column-specific basis.
    """
    df = (pd_df.replace
          (to_replace=dict_of_nonstandard_standard,
          value=None))
    return df

In [66]:
def standardise_headers(pd_df, dict_of_nonstandard_standard):
    """
    Changes the non-standard CSV column headers to harmonised-data 
    column headers. 
    e.g. 'Age Group' to 'Age'
    """
    # Do whitespaces need to be replaced with underscores?
    col_names = list(pd_df.columns)
    correction_dict = ({col_nm: col_nm.strip().title() 
                        for col_nm in col_names})
    df = pd_df.rename(columns=correction_dict)
    return df

In [67]:
def write_csv(pd_df, out_path):
    """
    Converts a Pandas dataframe to CSV and writes it out to a local folder.
        Parameters:
            pd_df (pd.Dataframe): The pandas data frame of the data
            path (string): the path of the local "out" folder
        Returns:
            writes csv """ #I am unsure how to express this output
    #code adapted from https://github.com/open-sdg/sdg-build/blob/797c7848d48de122d9eddfff6b2c8a9898f7e225/sdg/data.py#L35
#     def write_csv(inid, df, ftype='data', site_dir=''):
#     """
#     For a given ID and data set, write out as csv
#     Args:
#         inid: str. The indicator identifier
#         df: DataFrame. 
#         ftype: Sets directory path
#         site_dir: str. The site directory to build to.
#     Returns:
#         bool: Status
#     """
    status = True

    # If the csv dir isn't there, make it
    csv_dir = out_path
    if not os.path.exists(csv_dir):
        os.makedirs(csv_dir, exist_ok=True)

    try:
        df.to_csv(csv_dir, index=False)
    except Exception as e:
        print(e)
        return False

    return status                   


In [68]:
def entry_point(data_url):
    urls_gen = find_csv_urls(data_url)
    gap_filler_dict = (get_mapping_dicts(gap_filler_yam_path) 
    header_mapping_dict = (get_mapping_dicts(header_mapping_yam_path))

    for _url in urls_gen:
        data_name = extract_name(_url)
        df = csvs_to_pandas(_url)
        if not df:
            continue
        df = fill_gaps(df, gap_filler_dict)
        df = standardise_headers(df)
        write_csv(df, out_path)

if __name__ == "__main__":
    entry_point(data_url=remote_data_url)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\repo\\sdg_csvw\\sdg-csv-data-filler\\data\\gap_filler.yaml'

In [119]:

path
samp_filler_dict = get_mapping_dicts(path)

In [120]:
samp_df.fillna(value=samp_filler_dict, inplace=True)

In [191]:
def delete_random_values(df, holes=20):
    """
    Smashes holes in your dataframe to the approximate number that you
    request (randint might choose the same cell twice)
    """
    for i in range(holes):
        row = random.randint(1, samp_df.shape[0]-1)
        col = random.randint(0, samp_df.shape[1]-1)
        df.iloc[row, col] = float('nan')
    return df

In [187]:
samp_filler_dict

{'Year': 'forever',
 'Country': 'whole world',
 "Local Authority'": 'all the authorities',
 'Road category': 'yellow brick road',
 'Observation status': 'observing the unobservable',
 'Unit multiplier': 'infinity',
 'Unit measure': 'galactic years',
 'GeoCode': 'fourty two'}

In [195]:
test_yam_path = os.path.join(os.getcwd(), "substitutions", "samp_filler.yaml")
csv_path = os.path.join(os.getcwd(), "data", "indicator_9-1-1.csv")

def proof_of_concept(csv_path, path=test_yam_path):
    samp_df = csvsample_to_pandas(path_to_file=csv_path, pct=1)
    samp_filler_dict = get_mapping_dicts(path)
    holey_df = delete_random_values(samp_df)
    refilled_df = holey_df.fillna(value=samp_filler_dict)
    return refilled_df

poc_df = proof_of_concept(csv_path, path=test_yam_path)

In [196]:
poc_df

,Year,Country,Local Authority,Road category,Observation status,Unit multiplier,Unit measure,GeoCode,Value
0,2011,England,Telford and Wrekin,yellow brick road,Undefined,Units,Percentage (%),E06000020,100.000000
1,2011,England,Peterborough,yellow brick road,Undefined,Units,galactic years,E06000031,100.000000
2,2011,England,Brighton and Hove,yellow brick road,Undefined,Units,Percentage (%),fourty two,100.000000
3,2011,England,Worcester,yellow brick road,Undefined,Units,Percentage (%),E07000237,100.000000
4,2011,England,Uttlesford,Motorway,Undefined,infinity,Percentage (%),E07000077,33.068727
5,2011,England,Gedling,Motorway,Undefined,Units,Percentage (%),E07000173,0.000000
6,2011,England,York,A-road,Undefined,Units,Percentage (%),E06000014,70.223124
7,2011,England,Herefordshire,A-road,Undefined,Units,Percentage (%),E06000019,73.158640
8,2011,England,Central Bedfordshire,A-road,Undefined,Units,Percentage (%),E06000056,75.594670
9,2011,England,Kettering,A-road,Undefined,Units,Percentage (%),E07000153,87.616027
